# Sample Code

In [1]:
!apt-get -y install openjdk-8-jre-headless
!pip install pyspark

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Suggested packages:
  libnss-mdns fonts-dejavu-extra fonts-ipafont-gothic fonts-ipafont-mincho
  fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jre-headless
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 28.2 MB of archives.
After this operation, 104 MB of additional disk space will be used.
Ign:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 openjdk-8-jre-headless amd64 8u292-b10-0ubuntu1~18.04
Err:1 http://security.ubuntu.com/ubuntu bionic-updates/universe amd64 openjdk-8-jre-headless amd64 8u292-b10-0ubuntu1~18.04
  404  Not Found [IP: 91.189.88.142 80]
E: Failed to fetch http://security.ubuntu.com/ubuntu/pool/universe/o/openjdk-8/openjdk-8-jre-headless_8u292-b10-0ubuntu1~18.04_amd64.deb  404  Not Found [IP: 91.189.88.142 80]
E: Unable to fetch some archives, maybe run apt-get update o

In [2]:
from pyspark.sql import SparkSession, DataFrame, Window
from pyspark.sql.functions import col, regexp_extract, regexp_replace, lit, when
import pyspark.sql.functions as func
from pyspark.sql.types import *
from pyspark.sql.utils import AnalysisException
from pyspark import StorageLevel

import sys
from datetime import datetime, timedelta
from functools import reduce

In [3]:
spark = SparkSession.builder \
    .appName("content-based") \
    .config("spark.sql.files.ignoreCorruptFiles", "true") \
    .config("spark.sql.session.timeZone", "Asia/Taipei") \
    .getOrCreate()

## 基礎建設

In [4]:
import pandas as pd
import gzip, json

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

## 載入資料

In [5]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz

--2021-12-24 15:45:41--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15499476 (15M) [application/octet-stream]
Saving to: ‘All_Beauty.csv.1’

All_Beauty.csv.1    100%[===================>]  14.78M  19.1MB/s    in 0.8s    

2021-12-24 15:45:42 (19.1 MB/s) - ‘All_Beauty.csv.1’ saved [15499476/15499476]

--2021-12-24 15:45:42--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10329961 (9.9M) [application/octet-stream]
Saving to: ‘meta_All_Beauty.json.gz.1’

meta_All_Beauty.jso 100%[===================>]   9.85M  14.0MB/s    in 0.7s   

In [6]:
schema_rating = StructType([
                    StructField("asin", StringType())
                      , StructField("reviewerID", StringType())
                      , StructField("overall", StringType())
                      , StructField("unixReviewTime", StringType())
                      ])

In [7]:
metadata = spark.read.json("file:////content/meta_All_Beauty.json.gz")
ratings = spark.read.format("csv").option("header", True).schema(schema_rating).load("file:///content/All_Beauty.csv")

# metadata = getDF('/content/meta_All_Beauty.json.gz')
# ratings = pd.read_csv('/content/All_Beauty.csv', names=['asin', 'reviewerID', 'overall', 'unixReviewTime'], header=None)
# metadata.head()
# ratings.head()

## 資料整理

In [ ]:
#ratings['DATE'] = pd.to_datetime(ratings['unixReviewTime'], unit='s')

In [8]:
ratings = ratings.select("asin", "reviewerID", col("overall").cast(DoubleType()).alias("overall")
              , func.from_unixtime("unixReviewTime","yyyy-MM-dd HH:mm:ss").alias("timestamp"))

In [9]:
metadata = metadata.select(
                col("asin"), col("title"), col("feature"), col("description")
                , func.translate(col("price"), ',$', '').cast(DoubleType()).alias("price")
                , col("imageURL"), col("imageURLHighRes"), col("also_buy"), col("also_view")
                , func.translate(func.split(col("rank"), ' in ')[0], ',', '').cast(IntegerType()).alias("rank")
                , func.translate(func.split(col("rank"), ' in ')[1], '()"];', '').alias("category_for_rank")
                , col("brand"), col("tech1"), col("similar_item"), col("date")
                , col("details.\n    Item Weight: \n    ").alias("item_weight")
                , col("details.\n    Product Dimensions: \n    ").alias("product_dimensions")
                , col("details.Batteries").alias("batteries")
                , col("details.Discontinued by manufacturer:").alias("discountedByManufacturer")
                , col("details.Domestic Shipping: ").alias("domestic_shipping")
                , col("details.International Shipping: ").alias("international_shipping")
                , col("details.Item model number:").alias("item_model_no")
                , col("details.Shipping Advisory:").alias("shipping_advisory")
                , col("details.Shipping Weight:").alias("shipping_weight")
                )

In [31]:
# asin: do nothing on asin
# title: remove null, only one row
# feature: w feature -> 1, wo feature -> 2
# description: w desc -> 1, wo desc -> 2
metadata.select("asin", "title" 
                , func.when(func.size("feature") == 0, 2).otherwise(1).alias("feature_category") 
                , func.when(func.size("description") == 0, 2).otherwise(1).alias("description_category")
                , "price"
                , func.when(func.size("imageURL") == 0, 2).otherwise(1).alias("imageURL_category")
                , func.when(func.size("imageURLHighRes") == 0, 2).otherwise(1).alias("imageURLHighRes_category")
                , "also_buy", "also_view", "rank", "category_for_rank", "brand"
                , func.when(func.length("tech1") == 0, 2).otherwise(1).alias("tech1_category")
                , "similar_item", "date", "item_weight", "product_dimensions"
                , func.when(col("batteries").isNull(), 2).otherwise(1).alias("batteries_category")
                , func.when(col("discountedByManufacturer").isNull(), 2).otherwise(1).alias("discountedByManufacturer")
                , "domestic_shipping", "international_shipping", "item_model_no"
                , "shipping_advisory", "shipping_weight"
                ) \
        .filter(col("title") != '').show(2, False)

+----------+---------------------------------------------------------+----------------+--------------------+-----+-----------------+------------------------+------------------------+---------+-------+-----------------------+------------+--------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## 資料切分

In [ ]:
ratings_trainings = ratings[
    (ratings['DATE'] < '2018-09-01')
]
ratings_testings = ratings[
    (ratings['DATE'] >= '2018-09-01') & 
    (ratings['DATE'] <= '2018-09-30')
]
ratings_testings_by_user = ratings_testings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')
ratings_testings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_testings_by_user }
users = list(ratings_testings_by_user.keys())

## 產生推薦

In [ ]:
def recommender(training_data, users=[], k=10):
    '''
    * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
    * users: [] 需要被推薦的使用者
    * k: int 每個使用者需要推薦的商品數
    * recommendations: dict
      {
          使用者一： [推薦商品一, 推薦商品二, ...],
          使用者二： [...], ...
      }
    '''
    recommendations = {}
    '''
    Your Code
    '''
    return recommendations


ratings_by_user = recommender(ratings_trainings, users)
ratings_by_user

## 結果評估

In [ ]:
def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):
    '''
    * ratings_testings_by_user: dict 真實被購買的商品資料（2018-09-01 以後資料）
    * ratings_by_user: dict 利用訓練資料學習的推薦商品
    * method: str
    * score: float
    '''
    total = 0
    for d in ratings_testings_by_user:
        if d in ratings_by_user:
            total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))

    score = total / len(ratings_testings)
    return score

evaluate(ratings_testings_by_user, ratings_by_user)